In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import numpy as np
from ieeg.calc.stats import time_perm_cluster, window_averaged_shuffle
from ieeg.viz.mri import gen_labels

# from utils import make_or_load_subjects_electrodes_to_ROIs_dict, load_acc_arrays, calculate_RTs, save_channels_to_file, save_sig_chans, \
#       load_sig_chans, channel_names_to_indices, filter_and_average_epochs, permutation_test, perform_permutation_test_across_electrodes, perform_permutation_test_within_electrodes, \
#       add_accuracy_to_epochs, load_mne_objects, create_subjects_mne_objects_dict, extract_significant_effects, convert_dataframe_to_serializable_format, \
#       perform_modular_anova, make_plotting_parameters, plot_significance

import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
# still need to test if the permutation test functions load in properly.
import pandas as pd
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# from src.analysis.power.roi_analysis import blah_blah
from src.analysis.config import experiment_conditions
from src.analysis.config import plotting_parameters
import src.analysis.utils.general_utils as utils # import utils functions one by one by name
from src.analysis.power.power_traces import make_multi_channel_evokeds_for_all_conditions_and_rois, plot_power_traces_for_all_rois

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


get lab root for save dir

In [2]:
LAB_root = None
task='GlobalLocal'
# Determine LAB_root based on the operating system
if LAB_root is None:
    HOME = os.path.expanduser("~")
    LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir_root = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

choose which subjects you wanna run (has to be a list, even if just one subject)

In [3]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110', 'D0116', 'D0117', 'D0121']
# subjects = ['D0057']



### make or load subjects electrodes to rois dict

In [4]:
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
config_dir = os.path.join(project_root, 'src', 'analysis', 'config')

subjects_electrodestoROIs_dict = utils.make_or_load_subjects_electrodes_to_ROIs_dict(subjects, task='GlobalLocal', LAB_root=None, save_dir=config_dir, 
                                        filename='subjects_electrodestoROIs_dict.json')

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from c:\Users\jz421\Desktop\GlobalLocal\src\analysis\config\subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


check number of elecs

In [6]:
# Calculate the total number of electrodes in the 'default_dict' across subjects
total_electrodes = utils.count_electrodes_across_subjects(subjects_electrodestoROIs_dict, subjects)
print('total elecs:', total_electrodes)

total elecs: 3283


load behavioral data (combinedData.csv) and map blockType to congruency and switch proportions in the behavioral data

In [5]:
combined_data = pd.read_csv(os.path.join(LAB_root, 'D_Data', 'GlobalLocal', 'combinedData.csv'))
combined_data[['switchProportion', 'congruencyProportion']] = combined_data.apply(utils.map_block_type, axis=1)

### load epochs objects for chosen conditions for all subjects in a dictionary

In [8]:
task='GlobalLocal'
conditions = experiment_conditions.stimulus_err_corr_conditions # set this to whichever conditions you're running
stimulus_locked = True  #toggle
response_locked = not stimulus_locked

if stimulus_locked:
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_0.0-30.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False"
    # epochs_root_file = "Stimulus_100sec_within1-101sec_experimentStartBase_decFactor_8_outliers_10_passband_70-150_padLength_0.5s_stat_func_ttest_ind_equal_var_False"

elif response_locked:
    # epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_4.0-8.0_padLength_0.5s_stat_func_ttest_ind"
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind"

condition_names = [condition for condition in conditions.keys()]

In [9]:
if conditions == experiment_conditions.stimulus_conditions:
    conditions_save_name = 'stimulus_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_experiment_conditions:
    conditions_save_name = 'stimulus_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_main_effect_conditions:
    conditions_save_name = 'stimulus_main_effect_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwpc_conditions:
    conditions_save_name = 'stimulus_lwpc_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_lwps_conditions:
    conditions_save_name = 'stimulus_lwps_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_big_letter_conditions:
    conditions_save_name = 'stimulus_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_small_letter_conditions:
    conditions_save_name = 'stimulus_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_task_conditions:
    conditions_save_name = 'stimulus_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_congruency_conditions:
    conditions_save_name = 'stimulus_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_switch_type_conditions:
    conditions_save_name = 'stimulus_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.stimulus_err_corr_conditions:
    conditions_save_name = 'stimulus_err_corr_conditions_sem_green' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'

elif conditions == experiment_conditions.response_conditions:
    conditions_save_name = 'response_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_experiment_conditions:
    conditions_save_name = 'response_experiment_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_big_letter_conditions:
    conditions_save_name = 'response_big_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_small_letter_conditions:
    conditions_save_name = 'response_small_letter_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_task_conditions:
    conditions_save_name = 'response_task_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_congruency_conditions:
    conditions_save_name = 'response_congruency_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_switch_type_conditions:
    conditions_save_name = 'response_switch_type_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
elif conditions == experiment_conditions.response_err_corr_conditions:
    conditions_save_name = 'response_err_corr_conditions' + '_' + epochs_root_file + '_' + str(len(subjects)) + '_' + 'subjects'
    
# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
#subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True, error_trials_only=False)
#one without selecting only correct trials
subjects_mne_objects = utils.create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=False)

Loading data for subject: D0057
Reading /Users/erinburns/Library/CloudStorage/Box-Box/CoganLab/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/D0057/D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
449 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/erinburns/Library/CloudStorage/Box-Box/CoganLab/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/D0057/D0057_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_HG_ev1_power_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setti

/Users/erinburns/Desktop/GlobalLocal/src/analysis/utils/general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (32, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 40.1
    Max NaN count in a trial: 1282
    Stimulus_err: 32 valid trials out of 32
  Loading condition: Stimulus_corr with parameters: {'BIDS_events': ['Stimulus/Responded1.0/Accuracy1.0'], 'responseType': 'corr'}


/Users/erinburns/Desktop/GlobalLocal/src/analysis/utils/general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


Not setting metadata
405 matching events found
No baseline correction applied
    Original shape: (405, 175, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 121.9
    Max NaN count in a trial: 1923
    Stimulus_corr: 405 valid trials out of 405
  Loading condition: Stimulus_err with parameters: {'BIDS_events': ['Stimulus/Responded1.0/Accuracy0.0'], 'responseType': 'err'}
Not setting metadata
32 matching events found
No baseline correction applied


...

Not setting metadata
400 matching events found
No baseline correction applied
    Original shape: (400, 204, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 1123.4
    Max NaN count in a trial: 42947
    Stimulus_corr: 400 valid trials out of 400
  Loading condition: Stimulus_err with parameters: {'BIDS_events': ['Stimulus/Responded1.0/Accuracy0.0'], 'responseType': 'err'}
Not setting metadata
34 matching events found
No baseline correction applied


/Users/erinburns/Desktop/GlobalLocal/src/analysis/utils/general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


    Original shape: (34, 204, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 867.2
    Max NaN count in a trial: 8974
    Stimulus_err: 34 valid trials out of 34
  Loading condition: Stimulus_corr with parameters: {'BIDS_events': ['Stimulus/Responded1.0/Accuracy1.0'], 'responseType': 'corr'}


/Users/erinburns/Desktop/GlobalLocal/src/analysis/utils/general_utils.py:403: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  event_epochs = mne.concatenate_epochs(combined_epochs)


Not setting metadata
400 matching events found
No baseline correction applied
    Original shape: (400, 204, 641)
    Trials with all NaN values: 0
    Average NaN count per trial: 1123.4
    Max NaN count in a trial: 42947
    Stimulus_corr: 400 valid trials out of 400


### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [10]:
sig_chans_per_subject = utils.get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103
Loaded significant channels for subject D0107A
Loaded significant channels for subject D0110
Loaded significant channels for subject D0116
Loaded significant channels for subject D0117
Loaded significant channels for subject D0121


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [ ]:

# rois_dict = {
#     # 'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     # 'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
#     # 'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
#     'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
#     'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"]
# }

# the cns 24/sfn 24 poster plots need just one roi. Fix all this code later. 10/1.
# rois_dict = {
#     'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"]
# }

rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'dlpfc': ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'acc': ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"],
    'parietal': ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"],
}

rois = list(rois_dict.keys())
                                                
all_electrodes_per_subject_roi, sig_electrodes_per_subject_roi = make_sig_electrodes_per_subject_and_roi_dict(rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LAS

In [12]:
print('total sig elecs:', sum(len(sig_chans_per_subject[sub]) for sub in sig_chans_per_subject))

total sig elecs: 525


get number of sig and all electrodes per subject and across subjects

In [13]:
for roi in rois:
    for sub in subjects:
        sig_elecs = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        all_elecs = all_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        print(f"Subject {sub}, ROI {roi}, Num of Sig Electrodes: {len(sig_elecs)}, Num of All Electrodes: {len(all_elecs)}")

Subject D0057, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 17
Subject D0059, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 5
Subject D0063, ROI lpfc, Num of Sig Electrodes: 7, Num of All Electrodes: 36
Subject D0065, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 9
Subject D0069, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 1
Subject D0071, ROI lpfc, Num of Sig Electrodes: 3, Num of All Electrodes: 13
Subject D0077, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 0
Subject D0090, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 8
Subject D0094, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 28
Subject D0100, ROI lpfc, Num of Sig Electrodes: 0, Num of All Electrodes: 0
Subject D0102, ROI lpfc, Num of Sig Electrodes: 2, Num of All Electrodes: 5
Subject D0103, ROI lpfc, Num of Sig Electrodes: 8, Num of All Electrodes: 11
Subject D0107A, ROI lpfc, Num of Sig Electrodes: 1, Num of All Electrodes: 25
Subje

In [14]:
# Example usage:
total_electrodes_info = utils.calculate_total_electrodes(sig_electrodes_per_subject_roi, all_electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])

Total number of significant lpfc electrodes across all subjects: 44
Total number of lpfc electrodes across all subjects: 265
Total number of significant occ electrodes across all subjects: 18
Total number of occ electrodes across all subjects: 73
Total number of significant dlpfc electrodes across all subjects: 25
Total number of dlpfc electrodes across all subjects: 197
Total number of significant acc electrodes across all subjects: 8
Total number of acc electrodes across all subjects: 71
Total number of significant parietal electrodes across all subjects: 5
Total number of parietal electrodes across all subjects: 40


check sampling rates to make sure they're all the same

In [15]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = utils.check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)

Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.
Subject D0107A has the expected sampling rate: 256.0 Hz.
Subject D0110 has the expected sampling rate: 256.0 Hz.
Subject D0116 has the expected sampling rate: 256.0 Hz.
Subject D0117 has the expected sampling rate: 256.0 Hz.
Subject D0121 has the expected sampling rate: 256.0 Hz.


TODO: stats next (maybe do time perm cluster with t-test as follow-up test for ANOVA significance, let's try using MNE functions for ANOVA and perm test)   
TODO: add in code for selecting electrodes of a specific ROI  
TODO: plotting average traces

now let's actually try plotting

In [ ]:
save_dir = os.path.join(save_dir_root, epochs_root_file)

evks_dict_sig_elecs = make_multi_channel_evokeds_for_all_conditions_and_rois(
    subjects_mne_objects, subjects, rois, condition_names, 
    sig_electrodes_per_subject_roi
) 

evks_dict_all_elecs = make_multi_channel_evokeds_for_all_conditions_and_rois(
    subjects_mne_objects, subjects, rois, condition_names, 
    all_electrodes_per_subject_roi
)

plot_power_traces_for_all_rois(
    evks_dict_all_elecs, rois, condition_names, conditions_save_name, plotting_parameters,
    save_dir=save_dir,
    error_type='sem', figsize=(12, 8), 
    x_label='Time from Stimulus Onset (s)', 
    y_label='Power (z)',
    font_size=35, title_font_size=40, save_name_suffix='all_elecs'
)

plot_power_traces_for_all_rois(
    evks_dict_sig_elecs, rois, condition_names, conditions_save_name, plotting_parameters,
    save_dir=save_dir,
    error_type='sem', figsize=(12, 8), 
    x_label='Time from Stimulus Onset (s)', 
    y_label='Power (z)',
    font_size=35, title_font_size=40, save_name_suffix='sig_elecs'
)

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'LFAM9', 'LFAM8', 'RAI6'}. Applying running numbers for duplicates.
  info = mne.create_info(
/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'LFAM9', 'LFAM8'}. Applying running numbers for duplicates.
  info = mne.create_info(
/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'LFAM9', 'LFAM8', 'RAI6'}. Applying running numbers for duplicates.
  info = mne.create_info(
/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'LFAM9', 'LFAM8'}. Applying running numbers for duplicates.
  info = mne.create_info(


NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy functi

/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RAI12', 'LASF10', 'LFAI4', 'RIA15', 'RMSF11', 'RAMF11', 'LFAM9', 'RAMF14', 'LFMI5', 'RIA6', 'RAI6', 'LFO16', 'LFAM10', 'LFAM11', 'RIA5', 'RIA11', 'RIA13', 'LAI7', 'RFMM12', 'LFAM12', 'RAI15', 'RIA16', 'RAI16', 'LFO12', 'RIA14', 'LFO15', 'RASF16', 'LFO13', 'RFAI3', 'LFOP15', 'LFAM8', 'RMSF12', 'RAMF10', 'RFMM11', 'RAMF12', 'RIA12', 'RAI11', 'RFOA16', 'RFO14', 'RAI14', 'LAI8', 'RAMF13', 'RASF15'}. Applying running numbers for duplicates.
  info = mne.create_info(
/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RTPI2', 'RPIT1', 'RTPI1'}. Applying running numbers for duplicates.
  info = mne.create_info(
/Users/erinburns/Desktop/GlobalLocal/src/analysis/power/power_traces.py:38: RuntimeWarning: Channel names are not unique, found duplicates for: {'RA

Saved plot to: /Users/erinburns/Library/CloudStorage/Box-Box/CoganLab/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False/lpfc_stimulus_err_corr_conditions_sem_green_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_17_subjects_all_elecs_sem_shading.png
Saved plot to: /Users/erinburns/Library/CloudStorage/Box-Box/CoganLab/BIDS-1.1_GlobalLocal/BIDS/derivatives/freqFilt/figs/Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False/occ_stimulus_err_corr_conditions_sem_green_Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10_passband_70.0-150.0_padLength_0.5s_stat_func_ttest_ind_equal_var_False_17_subjects_all_elecs_sem_shading.p

In [17]:
import src.analysis.config.plotting_parameters as plotting_parameters

# This will print the exact path of the file Python is loading
print(plotting_parameters.__file__)

/Users/erinburns/Desktop/GlobalLocal/src/analysis/config/plotting_parameters.py
